In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_all_pages(base_url, max_pages):
    all_job_offers = []
    page_num = 1

    while page_num <= max_pages:
        url = f"{base_url}?page={page_num}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        job_offers = scrape_job_offers(soup)
        all_job_offers.extend(job_offers)
        
        page_num += 1

    return all_job_offers

def scrape_job_offers(soup):
    job_offers = []

    for job_card in soup.find_all('div', class_='employer-job-listing-single premium-job box-shadow bg-white mb-3 p-3 job-spotlight job-listing job_position_featured grid'):
        secteur_elem = job_card.find_all_next('p', class_='text-muted')
        sect = secteur_elem[1].text.split('·')[0]
        secteur = sect.strip() if secteur_elem else "N/A"
            
        contrat_elem = secteur_elem[1].text.split('·')[1]
        contrat = contrat_elem.strip() if contrat_elem != "Non spécifé" else "N/A"
            
        post_elem = job_card.find('h5')
        poste = post_elem.find('a', href=True)
        post = poste.text.strip() if poste else "N/A"
            
        company_elem = secteur_elem[2].text.split('·')[0]
        company = company_elem.strip() if company_elem else "N/A"
            
        link_elem = post_elem.find('a')
        link = link_elem.get('href')
        
        lieu_elem = secteur_elem[2].text.split('·')[1]
        lieux = lieu_elem.split(',')[0]
        lieu = lieux.strip() if lieux else "N/A"
            
        deadline_elem = secteur_elem[0].text.split('·')
        if len(deadline_elem) > 1: 
            deadlines = deadline_elem[1].strip()
            deadline  = deadlines.split('(')[1].strip()
            deadline = deadline.split(')')[0].strip()
        else:
            deadline = "N/A"

        job = {
            "Intitulé": post,
            "Entreprise": company,
            "Lien": link,
            "Secteur": secteur,
            "Type de contrat": contrat,
            "Lieu": lieu,
            "Deadline": deadline
        }
        
        job_offers.append(job)

    return job_offers

# Specify the base URL and the maximum number of pages to scrape
base_url = "https://jobs.doopinet.com/search"
max_pages = 100

# Scrape all pages and store the job details in a DataFrame
all_job_offers = scrape_all_pages(base_url, max_pages)
df = pd.DataFrame(all_job_offers)

# Convertissez la colonne de dates en format datetime
df['Deadline'] = pd.to_datetime(df['Deadline'], errors='coerce')

# Obtenez la date d'aujourd'hui
aujourdhui = datetime.today().date()

# Filtrez les lignes où la date est supérieure ou égale à la date d'aujourd'hui
df = df[df['Deadline'].dt.date > aujourdhui]

# Sauvegardez le DataFrame filtré dans un nouveau fichier CSV
df.to_csv('emplois_filtres.csv', index=False)
# 
# # Save the DataFrame to a CSV file
# df.to_csv('emplois.csv', index=False)
print("Les offres d'emploi ont été sauvegardées dans 'emplois_filtres.csv'.")




Les offres d'emploi ont été sauvegardées dans 'emplois_filtres.csv'.
